In [1]:
!pip install pandas mysql-connector-python streamlit plotly gitpython

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 25.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.7 MB/s eta 0:00:00
Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.3 MB/s eta 0:00:0031m39.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 30.1 MB/s eta 0:00:00
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
   ━━

In [2]:
import os

repo_url = "https://github.com/PhonePe/pulse.git"
clone_path = "phonepe_data"

if not os.path.exists(clone_path):
    os.system(f"git clone {repo_url} {clone_path}")
    print("Cloning done!")
else:
    print("Data already exists!")

Cloning into 'phonepe_data'...


Cloning done!


Updating files: 100% (9029/9029), done.


In [3]:
import json

file_path = "phonepe_data/data/aggregated/transaction/country/india/2018/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "from": 1514745000000,
    "to": 1522175400000,
    "transactionData": [
      {
        "name": "Recharge & bill payments",
        "paymentInstruments": [
          {
            "type": "TOTAL",
            "count": 72550406,
            "amount": 14472713558.652578
          }
        ]
      },
      {
        "name": "Peer-to-peer payments",
        "paymentInstruments": [
          {
            "type": "TOTAL",
            "count": 46982705,
            "amount": 147245883542.77402
          }
        ]
      },
      {
        "name": "Merchant payments",
        "paymentInstruments": [
          {
            "type": "TOTAL",
            "count": 5368669,
            "amount": 4656678915.140091
          }
        ]
      },
      {
        "name": "Financial Services",
        "paymentInstruments": [
          {
            "type": "TOTAL",
            "count": 3762820,
            "amount": 815853105.1000277
        

In [4]:
import os
import json
import pandas as pd

In [6]:
def extract_aggregated_transaction_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    transactions = data["data"]["transactionData"]
                    
                    for transaction in transactions:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "transaction_type": transaction["name"],
                            "transaction_count": transaction["paymentInstruments"][0]["count"],
                            "transaction_amount": transaction["paymentInstruments"][0]["amount"]
                        })
    
    return pd.DataFrame(records)

In [19]:
data_path = "phonepe_data/data/aggregated/transaction/country/india/state"

df_aggregated_transaction = extract_aggregated_transaction_data(data_path)

print(df_aggregated_transaction.shape)
print(df_aggregated_transaction.head())

(5034, 6)
                       state  year  quarter          transaction_type  \
0  andaman-&-nicobar-islands  2022        1     Peer-to-peer payments   
1  andaman-&-nicobar-islands  2022        1         Merchant payments   
2  andaman-&-nicobar-islands  2022        1  Recharge & bill payments   
3  andaman-&-nicobar-islands  2022        1        Financial Services   
4  andaman-&-nicobar-islands  2022        1                    Others   

   transaction_count  transaction_amount  
0             342649        1.682329e+09  
1             284298        3.700916e+08  
2             163401        7.130013e+07  
3               1757        2.869682e+06  
4               2643        2.440322e+06  


In [18]:
for item in os.listdir("phonepe_data/data/aggregated/transaction/country/india"):
    print(item)

2022
2024
2023
state
2019
2021
2020
2018


In [20]:
import sqlite3

conn = sqlite3.connect("phonepe_pulse.db")
print("Database connected successfully!")

Database connected successfully!


In [21]:
df_aggregated_transaction.to_sql(
    "aggregated_transaction",
    conn,
    if_exists="replace",
    index=False
)

print("Data saved successfully!")

Data saved successfully!


In [22]:
df_check = pd.read_sql("SELECT * FROM aggregated_transaction LIMIT 5", conn)
print(df_check)

                       state  year  quarter          transaction_type  \
0  andaman-&-nicobar-islands  2022        1     Peer-to-peer payments   
1  andaman-&-nicobar-islands  2022        1         Merchant payments   
2  andaman-&-nicobar-islands  2022        1  Recharge & bill payments   
3  andaman-&-nicobar-islands  2022        1        Financial Services   
4  andaman-&-nicobar-islands  2022        1                    Others   

   transaction_count  transaction_amount  
0             342649        1.682329e+09  
1             284298        3.700916e+08  
2             163401        7.130013e+07  
3               1757        2.869682e+06  
4               2643        2.440322e+06  


In [23]:
file_path = "phonepe_data/data/aggregated/user/country/india/state/andaman-&-nicobar-islands/2022/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "aggregated": {
      "registeredUsers": 76288,
      "appOpens": 3847407
    },
    "usersByDevice": [
      {
        "brand": "Vivo",
        "count": 18688,
        "percentage": 0.24496644295302014
      },
      {
        "brand": "Xiaomi",
        "count": 17957,
        "percentage": 0.23538433305369127
      },
      {
        "brand": "Samsung",
        "count": 12955,
        "percentage": 0.16981700922818793
      },
      {
        "brand": "Realme",
        "count": 6769,
        "percentage": 0.08872955117449664
      },
      {
        "brand": "Oppo",
        "count": 6461,
        "percentage": 0.08469221895973154
      },
      {
        "brand": "OnePlus",
        "count": 3005,
        "percentage": 0.039390205536912755
      },
      {
        "brand": "Tecno",
        "count": 1770,
        "percentage": 0.02320155201342282
      },
      {
        "brand": "Apple",
        "count": 1634,
        "percenta

In [24]:
def extract_aggregated_user_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    records.append({
                        "state": state,
                        "year": int(year),
                        "quarter": quarter,
                        "registered_users": data["data"]["aggregated"]["registeredUsers"],
                        "app_opens": data["data"]["aggregated"]["appOpens"]
                    })
    
    return pd.DataFrame(records)

In [25]:
user_data_path = "phonepe_data/data/aggregated/user/country/india/state"

df_aggregated_user = extract_aggregated_user_data(user_data_path)

print(df_aggregated_user.shape)
print(df_aggregated_user.head())

(1008, 5)
                       state  year  quarter  registered_users  app_opens
0  andaman-&-nicobar-islands  2022        1             76288    3847407
1  andaman-&-nicobar-islands  2022        2             80530    5180013
2  andaman-&-nicobar-islands  2022        3             84642    6068354
3  andaman-&-nicobar-islands  2022        4             88442    4354093
4  andaman-&-nicobar-islands  2024        1            108411   28538751


In [26]:
df_aggregated_user.to_sql(
    "aggregated_user",
    conn,
    if_exists="replace",
    index=False
)

print("User data saved!")

User data saved!


In [27]:
file_path = "phonepe_data/data/map/transaction/hover/country/india/state/andaman-&-nicobar-islands/2022/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "hoverDataList": [
      {
        "name": "north and middle andaman district",
        "metric": [
          {
            "type": "TOTAL",
            "count": 60059,
            "amount": 161624878.52274162
          }
        ]
      },
      {
        "name": "south andaman district",
        "metric": [
          {
            "type": "TOTAL",
            "count": 705512,
            "amount": 1872991195.902684
          }
        ]
      },
      {
        "name": "nicobars district",
        "metric": [
          {
            "type": "TOTAL",
            "count": 29177,
            "amount": 94414127.64974089
          }
        ]
      }
    ]
  },
  "responseTimestamp": 1651127589910
}


In [29]:
def extract_map_transaction_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    districts = data["data"]["hoverDataList"]
                    
                    for district in districts:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "district": district["name"],
                            "transaction_count": district["metric"][0]["count"],
                            "transaction_amount": district["metric"][0]["amount"]
                        })
    
    return pd.DataFrame(records)

In [30]:
map_transaction_path = "phonepe_data/data/map/transaction/hover/country/india/state"

df_map_transaction = extract_map_transaction_data(map_transaction_path)

print(df_map_transaction.shape)
print(df_map_transaction.head())

(20604, 6)
                       state  year  quarter  \
0  andaman-&-nicobar-islands  2022        1   
1  andaman-&-nicobar-islands  2022        1   
2  andaman-&-nicobar-islands  2022        1   
3  andaman-&-nicobar-islands  2022        2   
4  andaman-&-nicobar-islands  2022        2   

                            district  transaction_count  transaction_amount  
0  north and middle andaman district              60059        1.616249e+08  
1             south andaman district             705512        1.872991e+09  
2                  nicobars district              29177        9.441413e+07  
3  north and middle andaman district              72573        1.956932e+08  
4             south andaman district             954875        2.357327e+09  


In [31]:
df_map_transaction.to_sql(
    "map_transaction",
    conn,
    if_exists="replace",
    index=False
)

print("Map transaction data saved!")

Map transaction data saved!


In [32]:
file_path = "phonepe_data/data/map/user/hover/country/india/state/andaman-&-nicobar-islands/2022/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "hoverData": {
      "north and middle andaman district": {
        "registeredUsers": 10720,
        "appOpens": 904869
      },
      "south andaman district": {
        "registeredUsers": 63487,
        "appOpens": 2368756
      },
      "nicobars district": {
        "registeredUsers": 2081,
        "appOpens": 573782
      }
    }
  },
  "responseTimestamp": 1651127651992
}


In [34]:
def extract_map_user_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    for district_name, district_data in data["data"]["hoverData"].items():
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "district": district_name,
                            "registered_users": district_data["registeredUsers"],
                            "app_opens": district_data["appOpens"]
                        })
    
    return pd.DataFrame(records)

In [35]:
map_user_path = "phonepe_data/data/map/user/hover/country/india/state"

df_map_user = extract_map_user_data(map_user_path)

print(df_map_user.shape)
print(df_map_user.head())

(20608, 6)
                       state  year  quarter  \
0  andaman-&-nicobar-islands  2022        1   
1  andaman-&-nicobar-islands  2022        1   
2  andaman-&-nicobar-islands  2022        1   
3  andaman-&-nicobar-islands  2022        2   
4  andaman-&-nicobar-islands  2022        2   

                            district  registered_users  app_opens  
0  north and middle andaman district             10720     904869  
1             south andaman district             63487    2368756  
2                  nicobars district              2081     573782  
3  north and middle andaman district             11379     963824  
4             south andaman district             66959    3404740  


In [36]:
df_map_user.to_sql(
    "map_user",
    conn,
    if_exists="replace",
    index=False
)

print("Map user data saved!")

Map user data saved!


In [37]:
file_path = "phonepe_data/data/top/transaction/country/india/state/andaman-&-nicobar-islands/2022/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "states": null,
    "districts": [
      {
        "entityName": "south andaman",
        "metric": {
          "type": "TOTAL",
          "count": 705512,
          "amount": 1872991195.902684
        }
      },
      {
        "entityName": "north and middle andaman",
        "metric": {
          "type": "TOTAL",
          "count": 60059,
          "amount": 161624878.52274162
        }
      },
      {
        "entityName": "nicobars",
        "metric": {
          "type": "TOTAL",
          "count": 29177,
          "amount": 94414127.64974089
        }
      }
    ],
    "pincodes": [
      {
        "entityName": "744101",
        "metric": {
          "type": "TOTAL",
          "count": 209941,
          "amount": 615978080.6944855
        }
      },
      {
        "entityName": "744103",
        "metric": {
          "type": "TOTAL",
          "count": 157230,
          "amount": 399715980.1391735
        }
      },
  

In [39]:
def extract_top_transaction_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    for district in data["data"]["districts"]:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "entity_name": district["entityName"],
                            "entity_type": "district",
                            "transaction_count": district["metric"]["count"],
                            "transaction_amount": district["metric"]["amount"]
                        })
                    
                    for pincode in data["data"]["pincodes"]:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "entity_name": pincode["entityName"],
                            "entity_type": "pincode",
                            "transaction_count": pincode["metric"]["count"],
                            "transaction_amount": pincode["metric"]["amount"]
                        })
    
    return pd.DataFrame(records)

In [40]:
top_transaction_path = "phonepe_data/data/top/transaction/country/india/state"

df_top_transaction = extract_top_transaction_data(top_transaction_path)

print(df_top_transaction.shape)
print(df_top_transaction.head())

(18295, 7)
                       state  year  quarter               entity_name  \
0  andaman-&-nicobar-islands  2022        1             south andaman   
1  andaman-&-nicobar-islands  2022        1  north and middle andaman   
2  andaman-&-nicobar-islands  2022        1                  nicobars   
3  andaman-&-nicobar-islands  2022        1                    744101   
4  andaman-&-nicobar-islands  2022        1                    744103   

  entity_type  transaction_count  transaction_amount  
0    district             705512        1.872991e+09  
1    district              60059        1.616249e+08  
2    district              29177        9.441413e+07  
3     pincode             209941        6.159781e+08  
4     pincode             157230        3.997160e+08  


In [41]:
df_top_transaction.to_sql(
    "top_transaction",
    conn,
    if_exists="replace",
    index=False
)

print("Top transaction data saved!")

Top transaction data saved!


In [42]:
file_path = "phonepe_data/data/top/user/country/india/state/andaman-&-nicobar-islands/2022/1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

{
  "success": true,
  "code": "SUCCESS",
  "data": {
    "states": null,
    "districts": [
      {
        "name": "south andaman",
        "registeredUsers": 63487
      },
      {
        "name": "north and middle andaman",
        "registeredUsers": 10720
      },
      {
        "name": "nicobars",
        "registeredUsers": 2081
      }
    ],
    "pincodes": [
      {
        "name": "744103",
        "registeredUsers": 15011
      },
      {
        "name": "744105",
        "registeredUsers": 13070
      },
      {
        "name": "744101",
        "registeredUsers": 12300
      },
      {
        "name": "744102",
        "registeredUsers": 8271
      },
      {
        "name": "744202",
        "registeredUsers": 4107
      },
      {
        "name": "744107",
        "registeredUsers": 3455
      },
      {
        "name": "744104",
        "registeredUsers": 3146
      },
      {
        "name": "744112",
        "registeredUsers": 2552
      },
      {
        "name": "7

In [43]:
def extract_top_user_data(data_path):
    records = []
    
    for state in os.listdir(data_path):
        state_path = os.path.join(data_path, state)
        
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            
            for file in os.listdir(year_path):
                if file.endswith(".json"):
                    file_path = os.path.join(year_path, file)
                    quarter = int(file.replace(".json", ""))
                    
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    
                    for district in data["data"]["districts"]:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "entity_name": district["name"],
                            "entity_type": "district",
                            "registered_users": district["registeredUsers"]
                        })
                    
                    for pincode in data["data"]["pincodes"]:
                        records.append({
                            "state": state,
                            "year": int(year),
                            "quarter": quarter,
                            "entity_name": pincode["name"],
                            "entity_type": "pincode",
                            "registered_users": pincode["registeredUsers"]
                        })
    
    return pd.DataFrame(records)

In [44]:
top_user_path = "phonepe_data/data/top/user/country/india/state"

df_top_user = extract_top_user_data(top_user_path)

print(df_top_user.shape)
print(df_top_user.head())

(18296, 6)
                       state  year  quarter               entity_name  \
0  andaman-&-nicobar-islands  2022        1             south andaman   
1  andaman-&-nicobar-islands  2022        1  north and middle andaman   
2  andaman-&-nicobar-islands  2022        1                  nicobars   
3  andaman-&-nicobar-islands  2022        1                    744103   
4  andaman-&-nicobar-islands  2022        1                    744105   

  entity_type  registered_users  
0    district             63487  
1    district             10720  
2    district              2081  
3     pincode             15011  
4     pincode             13070  


In [45]:
df_top_user.to_sql(
    "top_user",
    conn,
    if_exists="replace",
    index=False
)

print("Top user data saved!")

Top user data saved!


In [46]:
tables = ["aggregated_transaction", "aggregated_user", 
          "map_transaction", "map_user",
          "top_transaction", "top_user"]

for table in tables:
    df = pd.read_sql(f"SELECT COUNT(*) as rows FROM {table}", conn)
    print(f"{table}: {df['rows'][0]} rows")

aggregated_transaction: 5034 rows
aggregated_user: 1008 rows
map_transaction: 20604 rows
map_user: 20608 rows
top_transaction: 18295 rows
top_user: 18296 rows


In [47]:
df_states_check = pd.read_sql("SELECT DISTINCT state FROM aggregated_transaction", conn)
print(df_states_check)

                                 state
0            andaman-&-nicobar-islands
1                           tamil-nadu
2                          lakshadweep
3                            telangana
4                              manipur
5                              haryana
6                              gujarat
7                               sikkim
8                                delhi
9                          west-bengal
10                       uttar-pradesh
11                                 goa
12                              punjab
13                   arunachal-pradesh
14                           karnataka
15                     jammu-&-kashmir
16                         maharashtra
17                              odisha
18                      madhya-pradesh
19                           rajasthan
20                      andhra-pradesh
21                          chandigarh
22                              kerala
23                        chhattisgarh
24                       

In [48]:
state_name_mapping = {
    "andaman-&-nicobar-islands": "Andaman & Nicobar Island",
    "andhra-pradesh": "Andhra Pradesh",
    "arunachal-pradesh": "Arunachal Pradesh",
    "assam": "Assam",
    "bihar": "Bihar",
    "chandigarh": "Chandigarh",
    "chhattisgarh": "Chhattisgarh",
    "dadra-&-nagar-haveli-&-daman-&-diu": "Dadra and Nagar Haveli",
    "delhi": "Delhi",
    "goa": "Goa",
    "gujarat": "Gujarat",
    "haryana": "Haryana",
    "himachal-pradesh": "Himachal Pradesh",
    "jammu-&-kashmir": "Jammu & Kashmir",
    "jharkhand": "Jharkhand",
    "karnataka": "Karnataka",
    "kerala": "Kerala",
    "ladakh": "Ladakh",
    "lakshadweep": "Lakshadweep",
    "madhya-pradesh": "Madhya Pradesh",
    "maharashtra": "Maharashtra",
    "manipur": "Manipur",
    "meghalaya": "Meghalaya",
    "mizoram": "Mizoram",
    "nagaland": "Nagaland",
    "odisha": "Odisha",
    "puducherry": "Puducherry",
    "punjab": "Punjab",
    "rajasthan": "Rajasthan",
    "sikkim": "Sikkim",
    "tamil-nadu": "Tamil Nadu",
    "telangana": "Telangana",
    "tripura": "Tripura",
    "uttar-pradesh": "Uttar Pradesh",
    "uttarakhand": "Uttarakhand",
    "west-bengal": "West Bengal"
}

print("Mapping created!")

Mapping created!
